<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Network_Representation_Python_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Network Representation - Python Exercise**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               color,                       # Color of the marker icon
               background_color,            # Background color of the marker icon
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=color,                       # Set the marker's color
                border_color=color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi'), default value = 'km'

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit ('km' or 'mi')
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/network_representation_exercise/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,lat,lon,demand,selling_price
0,C01,Concord,43.206898,-71.537994,9000,13
1,C02,Trenton,40.220596,-74.769913,10000,12
2,C03,Austin,30.274670,-97.740349,10000,10
3,C04,Providence,41.830914,-71.414963,9000,13
4,C05,Montpelier,44.262436,-72.580536,7000,15


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/network_representation/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()


,ID,name,lat,lon,supply
0,S01,Columbia,34.000343,-81.033211,40000
1,S02,Jefferson City,38.579201,-92.172935,30000
2,S03,Santa Fe,35.682240,-105.939728,20000
3,S04,Richmond,37.538857,-77.433640,32000
4,S05,Carson City,39.163914,-119.766121,20000


#### Definition of Classes

***❗Task 1a: modify the following code to account for a new attribute of customers, `selling_price`***

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon, demand):
        self.ID = ID              # Customer's ID
        self.name = name          # Customer's name
        self.lat = lat            # Customer's latitude
        self.lon = lon            # Customer's longitude
        self.demand = demand      # Customer's demand

In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, supply):
        self.ID = ID            # Supplier's ID
        self.name = name        # Supplier's name
        self.lat = lat          # Supplier's latitude
        self.lon = lon          # Supplier's longitude
        self.supply = supply    # Supplier's available supply


#### Creating node objects

In [ ]:
# Initializing an empty dictionary to store node objects
nodes = dict()

***❗Task 1b: modify the following code to account for a new attribute of customers, `selling_price`***

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'],         # Customer's longitude
                                    demand=row['demand'])   # Customer's demand

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'],         # Supplier's longitude
                                    supply=row['supply'])   # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**customers, **suppliers}

In [ ]:
# Verifying that the total supply is <= total demand

print('Total Supply:', sum([x.supply for x in suppliers.values()]))   # Calculate and print total supply
print('Total Demand:', sum([x.demand for x in customers.values()]))   # Calculate and print total demand

Total Supply: 171000
Total Demand: 171000


#### Visualizing node objects

In [ ]:
# Create a new map centered on the US with a zoom level of 5
map = folium.Map([40, -95.0], zoom_start=5)

# Plot customer locations with a warehouse icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='warehouse', color='green', background_color='yellow')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

***❗Task 2a: compute distances between suppliers and add them into the `distances` dictionary***

The following code snippet computes the distances between customers and suppliers, storing them in a dictionary called `distances`. Your task is to compute the distances between pairs of suppliers and add those values to the same `distances` dictionary.

*Hint: first loop through all suppliers using s1, supplier1. Then, inside that loop, go through all suppliers again using s2, supplier2.*

In [ ]:
# Creating a dictionary containing distances between customers and suppliers
distances = dict()
for s, supplier in suppliers.items():                                                                   # Iterate over suppliers
    for c, customer in customers.items():                                                               # Iterate over customers
        distances[s, c] = compute_geodesic_distance(origin=customer, destination=supplier, unit='km')   # Calculate and store distance in kilometers

# Your code here

***❗Task 2b: find a distance between supplier S01 and S04***

In [ ]:
# Your code here

#### Arc costs

In [ ]:
cost_unit_km = 1.2  # Cost per unit per kilometer

# Creating a dictionary containing unit costs between customers and suppliers
unit_cost = dict()
for s, supplier in suppliers.items():                                                           # Iterate over suppliers
    for c, customer in customers.items():                                                       # Iterate over customers
        unit_cost[s, c] = distances[s, c] * cost_unit_km                                        # Calculate unit cost as distance multiplied by cost per km


## **Analyzing the obtained network data**

In [ ]:
# Dictionary of all customers

customers

{'C01': <__main__.Customer at 0x7acfdc79a020>,
 'C02': <__main__.Customer at 0x7acfdc798f70>,
 'C03': <__main__.Customer at 0x7acfdc79ac20>,
 'C04': <__main__.Customer at 0x7acfdc79bc10>,
 'C05': <__main__.Customer at 0x7acfdc79be20>,
 'C06': <__main__.Customer at 0x7acfdc79a980>,
 'C07': <__main__.Customer at 0x7acfdc799b10>,
 'C08': <__main__.Customer at 0x7acfdc79bd90>,
 'C09': <__main__.Customer at 0x7acfdc79bdc0>,
 'C10': <__main__.Customer at 0x7acfdc79a680>,
 'C11': <__main__.Customer at 0x7acfdc711cc0>,
 'C12': <__main__.Customer at 0x7acfdc7112d0>,
 'C13': <__main__.Customer at 0x7acfdc711d80>,
 'C14': <__main__.Customer at 0x7acfdc711c00>,
 'C15': <__main__.Customer at 0x7acfdc7110c0>,
 'C16': <__main__.Customer at 0x7acfdc711390>,
 'C17': <__main__.Customer at 0x7acfdc7118d0>,
 'C18': <__main__.Customer at 0x7acfdc711540>,
 'C19': <__main__.Customer at 0x7acfdc711ae0>,
 'C20': <__main__.Customer at 0x7acfdc75c6d0>}

In [ ]:
# Accessing a specific customer object

customers['C01']


In [ ]:
# Accessing the demand information of a customer object

customers['C01'].demand

9000

In [ ]:
# Dictionary of all distances

distances

{('S01', 'C01'): 1312.8938802396121,
 ('S01', 'C02'): 886.2955026402644,
 ('S01', 'C03'): 1627.4628290579406,
 ('S01', 'C04'): 1211.1823805042338,
 ('S01', 'C05'): 1351.8400638458058,
 ('S01', 'C06'): 1001.3821449065863,
 ('S01', 'C07'): 1388.7909442652854,
 ('S01', 'C08'): 755.3070381113523,
 ('S01', 'C09'): 1038.0459520408672,
 ('S01', 'C10'): 1515.4187990236096,
 ('S01', 'C11'): 873.681466562938,
 ('S01', 'C12'): 2868.6035920809136,
 ('S01', 'C13'): 2237.002951450826,
 ('S01', 'C14'): 1424.0313994511068,
 ('S01', 'C15'): 786.3504987176916,
 ('S01', 'C16'): 576.7382275556878,
 ('S01', 'C17'): 684.2448665722762,
 ('S01', 'C18'): 786.6692041589342,
 ('S01', 'C19'): 2220.8141781170266,
 ('S01', 'C20'): 2188.1198796783424,
 ('S02', 'C01'): 1807.9104374273222,
 ('S02', 'C02'): 1507.6037327646927,
 ('S02', 'C03'): 1053.273188734065,
 ('S02', 'C04'): 1799.0891417592104,
 ('S02', 'C05'): 1749.9174415408538,
 ('S02', 'C06'): 256.1950777779769,
 ('S02', 'C07'): 355.9812160577384,
 ('S02', 'C08

In [ ]:
# Accessing specific distance

distances['S05', 'C01']

4014.9415990050666